In [14]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import os

def prompt_to_close_excel_files():
    messagebox.showinfo("Close Excel Files", "Please ensure all Excel files are closed before proceeding.")

def upload_file(title, filetypes):
    root = tk.Tk()
    root.withdraw()  # Hide the Tkinter GUI
    filepath = filedialog.askopenfilename(title=title, filetypes=filetypes)
    root.destroy()
    if filepath:
        messagebox.showinfo("File Uploaded", f"{title} uploaded successfully.")
    else:
        messagebox.showerror("File Upload Error", "No file was selected.")
    return filepath

def process_files(file_a_path, file_b_path):
    engine_a = 'xlrd' if file_a_path.endswith('.xls') else 'openpyxl'
    df_a = pd.read_excel(file_a_path, sheet_name='Template-New Participants', header=3, engine=engine_a)

    engine_b = 'xlrd' if file_b_path.endswith('.xls') else 'openpyxl'
    df_b = pd.read_excel(file_b_path, engine=engine_b)
    
    # Adjust the column names based on the new information
    filtered_df_b = df_b[df_b['PersNo'].isin(df_a['Personnel Number'])][['PersNo', 'BeforeTax_Rate', 'RothActualRt']]

    output_path = os.path.join(os.path.dirname(file_b_path), "filtered_data.xlsx")
    filtered_df_b.to_excel(output_path, index=False)
    messagebox.showinfo("Process Complete", f"Filtered data has been saved to {output_path}")
    
def main():
    prompt_to_close_excel_files()
    file_a_path = upload_file("Select the Personnel Numbers file (File A)", [("Excel files", "*.xlsx"), ("Excel 97-2003 files", "*.xls")])
    if file_a_path:
        file_b_path = upload_file("Select the Alight data file (File B)", [("Excel files", "*.xlsx"), ("Excel 97-2003 files", "*.xls")])
        if file_b_path:
            process_files(file_a_path, file_b_path)
        else:
            print("File B was not selected.")
    else:
        print("File A was not selected.")

if __name__ == "__main__":
    main()
